In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import time
import datetime as dt
import os
import seaborn as sns
import scipy.stats as sts

import random
import string

import polars as pl

In [ ]:
from nb_vars import FOLDER_INTERMEDIATE, FOLDER_OUTPUT, CUTOFF_SPIKES_HZ

## Load df of filtered peaks

In [ ]:
CONDITION_NAME = "EXPERIMENTO"
CONDITION = "1"  

folder_input =  f'{FOLDER_INTERMEDIATE}/{CONDITION_NAME} {CONDITION}'

df_peaks_exp_1 = pl.read_csv(f'{folder_input}/df_peaks_full_{CONDITION_NAME}_{CONDITION}_freq_{CUTOFF_SPIKES_HZ}.csv').to_pandas()


In [ ]:
CONDITION_NAME = "CONTROL"
CONDITION = "1" 

folder_input =  f'{FOLDER_INTERMEDIATE}/{CONDITION_NAME} {CONDITION}'

df_peaks_ctrl_1 = pl.read_csv(f'{folder_input}/df_peaks_full_{CONDITION_NAME}_{CONDITION}_freq_{CUTOFF_SPIKES_HZ}.csv').to_pandas()

In [ ]:
CONDITION_NAME = "EXPERIMENTO"
CONDITION = "2" 

folder_input =  f'{FOLDER_INTERMEDIATE}/{CONDITION_NAME} {CONDITION}'

df_peaks_exp_2 = pl.read_csv(f'{folder_input}/df_peaks_full_{CONDITION_NAME}_{CONDITION}_freq_{CUTOFF_SPIKES_HZ}.csv').to_pandas()


In [ ]:
CONDITION_NAME = "CONTROL"
CONDITION = "2" 

folder_input =  f'{FOLDER_INTERMEDIATE}/{CONDITION_NAME} {CONDITION}'

df_peaks_ctrl_2 = pl.read_csv(f'{folder_input}/df_peaks_full_{CONDITION_NAME}_{CONDITION}_freq_{CUTOFF_SPIKES_HZ}.csv').to_pandas()

In [ ]:
def adapt_MFR_df(df):
    df['day'] = [1 if '24h' in i else 0 for i in df['time_exp']]
    df.loc[df['time_exp'] == 'Base', 'day'] = -1 
    df['min'] = [int(i.replace('24h + ', '').replace(' min', '')) if i != 'Base' else 0 for i in df['time_exp']]

    df = df.sort_values(by=['day', 'min'])

    df['condition_time'] = df['condition'] + ' | ' + df['time_exp']
    df['condition_time'] = df['condition_time'].apply(lambda x: x.replace(' min', '').replace('h + ', '_'))

    df['time_exp'] = df['time_exp'].apply(lambda x: x.replace(' min', '´'))

    return df

## Relative change in MFR (without Baseline normalisation)

### EXPERIMENT 1

In [ ]:
df_MFR_ctrl_1 = df_peaks_ctrl_1.groupby(['condition', 'time_exp', 'well', 'replicate', 'electrode']).count()['time'] / (df_peaks_ctrl_1['time'].max() - df_peaks_ctrl_1['time'].min())
df_MFR_ctrl_1 = df_MFR_ctrl_1.reset_index().sort_values(by=['condition', 'time_exp','well', 'electrode'])
df_MFR_ctrl_1 = df_MFR_ctrl_1.rename(columns={'time': 'MFR'})

df_MFR_ctrl_1 = adapt_MFR_df(df_MFR_ctrl_1)
df_MFR_ctrl_1

fig, ax = plt.subplots(1,1, figsize=(28,6))
sns.boxplot(data=df_MFR_ctrl_1, x='time_exp', y='MFR', hue = 'replicate', ax=ax)


In [ ]:
df_MFR_exp_1 = df_peaks_exp_1.groupby(['condition', 'time_exp', 'well', 'replicate', 'electrode']).count()['time'] / (df_peaks_exp_1['time'].max() - df_peaks_exp_1['time'].min())
df_MFR_exp_1 = df_MFR_exp_1.reset_index().sort_values(by=['condition', 'time_exp','well', 'electrode'])
df_MFR_exp_1 = df_MFR_exp_1.rename(columns={'time': 'MFR'})

df_MFR_exp_1 = adapt_MFR_df(df_MFR_exp_1)
df_MFR_exp_1

In [ ]:
df_MFR_ctrl_exp_1 = pd.concat([df_MFR_ctrl_1, df_MFR_exp_1])

fig, ax = plt.subplots(1,1, figsize=(28,6))
sns.boxplot(data=df_MFR_ctrl_exp_1, x='time_exp', y='MFR', hue = 'condition', ax=ax)
plt.show()


In [ ]:
for tto in ['H PLKO', 'H 219', 'LIPOS -', 'LIPOS 219']:
    fig, ax = plt.subplots(1,1, figsize=(28,6))
    sns.boxplot(data=df_MFR_exp_1, x='time_exp', y='MFR', ax=ax)
    ax.set_title(tto + ' MERGED REPLICATES')
    plt.show()

    fig, ax = plt.subplots(1,1, figsize=(28,6))
    sns.boxplot(data=df_MFR_exp_1, x='time_exp', y='MFR', ax=ax, hue='replicate')
    ax.set_title(tto + ' INDIVIDUAL REPLICATES')
    plt.show()

### EXPERIMENT 2

In [ ]:
df_MFR_ctrl_2 = df_peaks_ctrl_2.groupby(['condition', 'time_exp', 'well', 'replicate', 'electrode']).count()['time'] / (df_peaks_ctrl_2['time'].max() - df_peaks_ctrl_2['time'].min())
df_MFR_ctrl_2 = df_MFR_ctrl_2.reset_index().sort_values(by=['condition', 'time_exp','well', 'electrode'])
df_MFR_ctrl_2 = df_MFR_ctrl_2.rename(columns={'time': 'MFR'})

df_MFR_ctrl_2 = adapt_MFR_df(df_MFR_ctrl_2)
df_MFR_ctrl_2

fig, ax = plt.subplots(1,1, figsize=(28,6))
sns.boxplot(data=df_MFR_ctrl_2, x='time_exp', y='MFR', hue = 'replicate', ax=ax)

In [ ]:
df_MFR_exp_2 = df_peaks_exp_2.groupby(['condition', 'time_exp', 'well', 'replicate', 'electrode']).count()['time'] / (df_peaks_exp_2['time'].max() - df_peaks_exp_2['time'].min())
df_MFR_exp_2 = df_MFR_exp_2.reset_index().sort_values(by=['condition', 'time_exp','well', 'electrode'])
df_MFR_exp_2 = df_MFR_exp_2.rename(columns={'time': 'MFR'})

df_MFR_exp_2 = adapt_MFR_df(df_MFR_exp_2)
df_MFR_exp_2

In [ ]:
df_MFR_ctrl_exp_2 = pd.concat([df_MFR_ctrl_2, df_MFR_exp_2])

fig, ax = plt.subplots(1,1, figsize=(28,6))
sns.boxplot(data=df_MFR_ctrl_exp_2, x='time_exp', y='MFR', hue = 'condition', ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(28,6))
sns.boxplot(data=df_MFR_ctrl_2, x='time_exp', y='MFR', hue = 'replicate', ax=ax)
plt.show()

In [ ]:
df_MFR_2 = df_peaks_exp_2.groupby(['condition', 'time_exp', 'well', 'replicate', 'electrode']).count()['time'] / (df_peaks_exp_2['time'].max() - df_peaks_exp_2['time'].min())
df_MFR_2 = df_MFR_2.reset_index().sort_values(by=['well', 'electrode'])
df_MFR_2 = df_MFR_2.rename(columns={'time': 'MFR'})

df_MFR_2 = adapt_MFR_df(df_MFR_2)
df_MFR_2

In [ ]:
for tto in ['GLIO PLKO', 'GLIO 219', 'LIPOS -', 'ASTRO 219']:
    df_MFR_2_sub = df_MFR_2[df_MFR_2['condition'].isin(['BASAL', tto])]
    df_MFR_2_sub.loc[df_MFR_2_sub['condition'] == 'BASAL', 'time_exp'] = 'B'

    print(tto)
    fig, ax = plt.subplots(1,1, figsize=(28,6))
    sns.boxplot(data=df_MFR_2_sub, x='time_exp', y='MFR', ax=ax)
    ax.set_title(tto + ' MERGED REPLICATES')
    plt.show()

    fig, ax = plt.subplots(1,1, figsize=(28,6))
    sns.boxplot(data=df_MFR_2_sub, x='time_exp', y='MFR', ax=ax, hue='replicate')
    ax.set_title(tto + ' INDIVIDUAL REPLICATES')
    plt.show()

## Relative change in MFR (with Baseline normalisation)

In [ ]:
def normalize_MFR(df, condition):
    df_sub = df[df['condition'] == condition]
    base_df = df_sub[df_sub['time_exp'] == 'Base']
    rest_df = df_sub[df_sub['time_exp'] != 'Base']

    norm_df = rest_df.copy()
    norm_df = norm_df.rename(columns={'MFR': 'MFR_norm'})

    list_wells_electrodes = base_df[['well', 'electrode']].values

    for well, electrode in zip(list_wells_electrodes.T[0],list_wells_electrodes.T[1]):
        MFR_top = rest_df.loc[(rest_df['well'] == well) & (rest_df['electrode'] == electrode), 'MFR'].values
        MFR_base = base_df.loc[(base_df['well'] == well) & (base_df['electrode'] == electrode), 'MFR'].values

        MFR_norm = (MFR_top / MFR_base)**0.5  # This helps reduce some noise in the data

        norm_df.loc[(norm_df['well'] == well) & (norm_df['electrode'] == electrode), 'MFR_norm'] = MFR_norm

    return norm_df

### EXPERIMENT 1

In [ ]:
df_MFR_ctrl_1_norm = normalize_MFR(df_MFR_ctrl_1, condition='CONTROL')

fig, ax = plt.subplots(1,1, figsize=(28,6))
sns.boxplot(data=df_MFR_ctrl_1_norm, x='time_exp', y='MFR_norm', hue = 'replicate', ax=ax)
ax.set_ylim([0, 2.5])

In [ ]:
df_MFR_exp_1_norm_list = [normalize_MFR(df_MFR_exp_1, condition=condition) for condition in df_MFR_exp_1['condition'].drop_duplicates().values]
df_MFR_exp_1_norm = pd.concat(df_MFR_exp_1_norm_list)

df_MFR_ctrl_exp_1_norm = pd.concat([df_MFR_ctrl_1_norm, df_MFR_exp_1_norm])


fig, ax = plt.subplots(1,1, figsize=(28,6))
sns.boxplot(data=df_MFR_ctrl_exp_1_norm, x='time_exp', y='MFR_norm', hue = 'condition', ax=ax)
ax.set_ylim([0, 3])
plt.show()


In [ ]:
for tto in df_MFR_ctrl_exp_1_norm['condition'].drop_duplicates().values:
    fig, ax = plt.subplots(1,1, figsize=(28,6))
    sns.boxplot(data=df_MFR_ctrl_exp_1_norm, x='time_exp', y='MFR_norm', ax=ax)
    ax.set_title(tto + ' MERGED REPLICATES')
    ax.set_ylim([0, 3])
    plt.show()

    fig, ax = plt.subplots(1,1, figsize=(28,6))
    sns.boxplot(data=df_MFR_ctrl_exp_1_norm, x='time_exp', y='MFR_norm', ax=ax, hue='replicate')
    ax.set_title(tto + ' INDIVIDUAL REPLICATES')
    ax.set_ylim([0, 3])
    plt.show()

### EXPERIMENT 2

In [ ]:
df_MFR_ctrl_2_norm = normalize_MFR(df_MFR_ctrl_2, condition='CONTROL')

fig, ax = plt.subplots(1,1, figsize=(28,6))
sns.boxplot(data=df_MFR_ctrl_2_norm, x='time_exp', y='MFR_norm', hue = 'replicate', ax=ax)
ax.set_ylim([0, 2])

In [ ]:
df_MFR_exp_2_norm_list = [normalize_MFR(df_MFR_exp_2, condition=condition) for condition in df_MFR_exp_2['condition'].drop_duplicates().values]
df_MFR_exp_2_norm = pd.concat(df_MFR_exp_2_norm_list)

df_MFR_ctrl_exp_2_norm = pd.concat([df_MFR_ctrl_2_norm, df_MFR_exp_2_norm])


fig, ax = plt.subplots(1,1, figsize=(28,6))
sns.boxplot(data=df_MFR_ctrl_exp_2_norm, x='time_exp', y='MFR_norm', hue = 'condition', ax=ax)
ax.set_ylim([0, 3])
plt.show()

In [ ]:
for tto in df_MFR_ctrl_exp_2_norm['condition'].drop_duplicates().values:
    fig, ax = plt.subplots(1,1, figsize=(28,6))
    sns.boxplot(data=df_MFR_ctrl_exp_2_norm, x='time_exp', y='MFR_norm', ax=ax)
    ax.set_title(tto + ' MERGED REPLICATES')
    plt.show()

    fig, ax = plt.subplots(1,1, figsize=(28,6))
    sns.boxplot(data=df_MFR_ctrl_exp_2_norm, x='time_exp', y='MFR_norm', ax=ax, hue='replicate')
    ax.set_title(tto + ' INDIVIDUAL REPLICATES')
    plt.show()

## Plotting Exp1 and Exp2 together

In [ ]:
df_MFR_ctrl_exp_12_norm = pd.concat([df_MFR_ctrl_exp_1_norm, df_MFR_ctrl_exp_2_norm])
df_MFR_ctrl_exp_12_norm['experiment'] = ['1'] * len(df_MFR_ctrl_exp_1_norm) + ['2'] * len(df_MFR_ctrl_exp_2_norm)

df_MFR_ctrl_exp_12_norm['condition_experiment'] = df_MFR_ctrl_exp_12_norm['condition'] + ' ' + df_MFR_ctrl_exp_12_norm['experiment']

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(35,6))
sns.boxplot(data=df_MFR_ctrl_exp_12_norm, x='time_exp', y='MFR_norm', hue = 'condition_experiment', ax=ax)
ax.set_ylim([0, 3])
plt.show()

**Conclusions derived from the exp 1+2 analysis with baseline normalization**

- Exp 1 and Exp 2 are not directly comparable, although both suffer (in exp 1 is more pronounced) a decline in signal at the beginning, which tends to neutralize towards the end, and keeps at the same rate at 24h (approx).
- Respective to their experiments, lipos are the condition that most decrease the MFR.
- In Exp 1, lipos 219 shows higher decrease than lipos, but H 219 is increased compared to H PLKO.
- In Exp 2, GLIO PLKO and 219 show a similar effect.